## 가설검정의 이해

### 관련 라이브러리 호출

In [1]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
# 통계 관련 라이브러리를 호출합니다.
from scipy import stats
import pingouin as pg
import scikit_posthocs as sp
from statsmodels.stats.proportion import proportions_ztest

### 작업 경로 확인 및 변경

In [3]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

'/Users/hdsceokevin/Documents/Lectures/PythonAdvanced/code'

In [4]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [5]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

['Used_Cars_Price.z',
 'Used_Cars_Price.xlsx',
 'Seafood_Trade_Prep.z',
 'Used_Cars_Price_Prep.z',
 'Used_Cars_Price.csv']

### 실습 데이터셋 준비

In [6]:
# z 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price.z')

In [7]:
# df의 정보를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1268 non-null   int64 
 1   Age        1268 non-null   int64 
 2   KM         1268 non-null   int64 
 3   FuelType   1268 non-null   object
 4   HP         1268 non-null   int64 
 5   MetColor   1268 non-null   object
 6   Automatic  1268 non-null   object
 7   CC         1268 non-null   int64 
 8   Doors      1268 non-null   int64 
 9   Weight     1268 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 99.2+ KB


In [8]:
# df의 처음 5행을 출력합니다.
df.head()

,Price,Age,KM,FuelType,HP,MetColor,Automatic,CC,Doors,Weight
0,13500,23,46986,Diesel,90,1,0,2000,3,1165
1,13750,23,72937,Diesel,90,1,0,2000,3,1165
2,13950,24,41711,Diesel,90,1,0,2000,3,1165
3,14950,26,48000,Diesel,90,0,0,2000,3,1165
4,13750,30,38500,Diesel,90,0,0,2000,3,1170


### 공분산

In [9]:
# 두 연속형 변수의 공분산을 반환합니다.
df['Age'].cov(df['Price'])

-22157.692905818414

In [10]:
# df의 연속형 변수로 분산-공분산 행렬을 반환합니다.
df.cov(numeric_only = True)

,Price,Age,KM,HP,CC,Doors,Weight
Price,4.117236e+06,-22157.692906,-3.744107e+07,5899.685571,1.880875e+04,317.011285,15155.221834
Age,-2.215769e+04,187.941609,1.711871e+05,-7.990297,-2.136068e+02,-1.180889,-98.225705
KM,-3.744107e+07,171187.124497,1.285820e+09,-155996.120014,2.557935e+06,595.872724,359258.457561
HP,5.899686e+03,-7.990297,-1.559961e+05,171.607575,-4.668580e+01,1.496539,-39.090577
CC,1.880875e+04,-213.606801,2.557935e+06,-46.685804,3.373000e+04,23.854276,5001.292993
Doors,3.170113e+02,-1.180889,5.958727e+02,1.496539,2.385428e+01,0.899761,13.211410
Weight,1.515522e+04,-98.225705,3.592585e+05,-39.090577,5.001293e+03,13.211410,1145.078508


### 상관계수

In [11]:
# 두 연속형 변수의 피어슨 상관계수를 반환합니다.
df['Age'].corr(df['Price'])

-0.7965447290519274

In [12]:
# df의 연속형 변수로 피어슨 상관계수 행렬을 반환합니다.
df.corr(numeric_only = True)

,Price,Age,KM,HP,CC,Doors,Weight
Price,1.000000,-0.796545,-0.514583,0.221951,0.050472,0.164706,0.220720
Age,-0.796545,1.000000,0.348233,-0.044492,-0.084839,-0.090810,-0.211737
KM,-0.514583,0.348233,1.000000,-0.332090,0.388411,0.017519,0.296073
HP,0.221951,-0.044492,-0.332090,1.000000,-0.019405,0.120436,-0.088183
CC,0.050472,-0.084839,0.388411,-0.019405,1.000000,0.136929,0.804742
Doors,0.164706,-0.090810,0.017519,0.120436,0.136929,1.000000,0.411593
Weight,0.220720,-0.211737,0.296073,-0.088183,0.804742,0.411593,1.000000


### 피어슨 상관분석

In [13]:
# Age와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Age'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,-0.796545,"[-0.82, -0.78]",5.879611e-279,6.169e+274,1.0


In [14]:
# KM와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['KM'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,-0.514583,"[-0.55, -0.47]",1.187373e-86,9.482e+82,1.0


In [15]:
# HP와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['HP'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,0.221951,"[0.17, 0.27]",1.289468e-15,2.583e+12,1.0


In [16]:
# CC와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['CC'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,0.050472,"[-0.0, 0.11]",0.072396,0.176,0.435457


In [17]:
# Doors와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Doors'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,0.164706,"[0.11, 0.22]",3.644964e-09,1.245e+06,0.999962


In [18]:
# Weight와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Weight'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,0.22072,"[0.17, 0.27]",1.863429e-15,1.798e+12,1.0


### [참고] 피어슨 상관분석 유의확률 출력 함수 생성

In [19]:
# 변수 x에 df의 연속형 변수를 할당합니다.
x = df['Age']

In [20]:
# 변수 x와 Price의 피어슨 상관분석 실행 결과에서 유의확률만 출력합니다.
pg.corr(x = x, y = df['Price'])['p-val']

pearson    5.879611e-279
Name: p-val, dtype: float64

In [21]:
# 연속형 입력변수와의 상관분석 유의확률을 출력하는 람다 표현식을 생성합니다.
corr = lambda x: pg.corr(x = x, y = df['Price'])['p-val']

In [22]:
# 람다 표현식 함수로 피어슨 상관분석 유의확률을 출력합니다.
corr(x = df['Age'])

pearson    5.879611e-279
Name: p-val, dtype: float64

### [참고] apply() 함수를 활용한 상관분석 실행

In [23]:
# df의 열별 자료형을 확인합니다.
df.dtypes

Price         int64
Age           int64
KM            int64
FuelType     object
HP            int64
MetColor     object
Automatic    object
CC            int64
Doors         int64
Weight        int64
dtype: object

In [24]:
# 열별 자료형이 정수형 또는 실수형이면 True, 아니면 False인 벡터를 생성합니다.
locs = df.dtypes.astype(str).isin(values = ['float64', 'int64'])
locs

Price         True
Age           True
KM            True
FuelType     False
HP            True
MetColor     False
Automatic    False
CC            True
Doors         True
Weight        True
dtype: bool

In [25]:
# df의 정수형 또는 실수형 변수만 선택하여 상관분석을 실행하고, 
# 유의확률이 0.05보다 작은지 여부를 데이터프레임으로 반환합니다.
df.loc[:, locs].apply(func = corr).lt(0.05)

,Price,Age,KM,HP,CC,Doors,Weight
pearson,True,True,True,True,False,True,True


### t-검정: MetColor

In [26]:
# MetColor 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'MetColor', method = 'shapiro')

,W,pval,normal
MetColor,,,
1,0.974759,7.077313e-11,False
0,0.988073,1.429560e-03,False


In [27]:
# (정규성 가정 만족) MetColor 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'MetColor', method = 'levene')

,W,pval,equal_var
levene,5.761315,0.016526,False


In [28]:
# MetColor 범주별 Price로 시리즈를 생성합니다.
sp1 = df['Price'][df['MetColor'].eq('0')]
sp2 = df['Price'][df['MetColor'].eq('1')]

In [29]:
# 두 시리즈의 평균을 확인합니다.
print(sp1.mean())
print(sp2.mean())

9466.672897196262
9814.132142857143


In [30]:
# 등분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.891752,1266,two-sided,0.003896,"[-583.18, -111.73]",0.171735,4.077,0.82371


In [31]:
# 이분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.983633,935.893141,two-sided,0.002922,"[-576.0, -118.92]",0.171735,5.313,0.82371


In [32]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = sp1, y = sp2)

,U-val,alternative,p-val,RBC,CLES
MWU,166376.0,two-sided,0.029849,0.074455,0.462773


### t-검정: Automatic

In [33]:
# Automatic 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'Automatic', method = 'shapiro')

,W,pval,normal
Automatic,,,
0,0.981458,2.977123e-11,False
1,0.948419,5.997743e-03,False


In [34]:
# (정규성 가정 만족) Automatic 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'Automatic', method = 'levene')

,W,pval,equal_var
levene,1.092962,0.296015,True


In [35]:
# Automatic 범주별 Price로 시리즈를 생성합니다.
sp1 = df['Price'][df['Automatic'].eq('0')]
sp2 = df['Price'][df['Automatic'].eq('1')]

In [36]:
# 두 시리즈의 평균을 확인합니다.
print(sp1.mean())
print(sp2.mean())

9676.575125208681
10043.857142857143


In [37]:
# 등분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.472702,1266,two-sided,0.14108,"[-856.55, 121.99]",0.181091,0.377,0.312941


In [38]:
# 이분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.567689,78.571012,two-sided,0.120972,"[-833.65, 99.08]",0.181091,0.433,0.312941


In [39]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = sp1, y = sp2)

,U-val,alternative,p-val,RBC,CLES
MWU,37216.5,two-sided,0.113249,0.112414,0.443793


### 모평균 검정

In [40]:
# 단일표본, 대응표본 t-검정에 사용할 데이터프레임 df1을 생성합니다.
df1 = pd.read_csv(filepath_or_buffer = 'https://bit.ly/ttest_dataset')

In [41]:
# df1의 정보를 확인합니다.
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      30 non-null     int64  
 1   before  30 non-null     float64
 2   after   30 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 848.0 bytes


In [42]:
# df1의 처음 5행을 출력합니다.
df1.head()

,id,before,after
0,1,88.409617,89.047771
1,2,84.740057,83.796276
2,3,106.925242,105.782969
3,4,93.276659,89.965632
4,5,104.575636,103.410691


In [43]:
# df1 수치형 변수의 기술통계량을 확인합니다.
df1.describe()

,id,before,after
count,30.000000,30.000000,30.000000
mean,15.500000,86.331409,85.325118
std,8.803408,9.918105,9.779472
min,1.000000,67.503561,65.462936
25%,8.250000,80.123625,79.998144
50%,15.500000,86.617928,86.149042
75%,22.750000,91.690221,90.259867
max,30.000000,106.925242,105.782969


In [44]:
# before의 정규성 검정을 실행합니다.
stats.shapiro(x = df1['before'])[1]

0.9512473940849304

In [45]:
# 단일표본 t-검정을 실행합니다.(양측검정)
pg.ttest(x = df1['before'], y = 90)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.025961,29,two-sided,0.052056,"[82.63, 90.03]",0.369888,1.159,0.499536


In [46]:
# 단일표본 t-검정을 실행합니다.(단측검정)
pg.ttest(x = df1['before'], y = 90, alternative = 'less')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.025961,29,less,0.026028,"[-inf, 89.41]",0.369888,2.318,0.63062


In [47]:
# df1의 열별 정규성 검정을 실행합니다.
df1.apply(func = lambda x: stats.shapiro(x)[1])

id        0.266230
before    0.951247
after     0.939600
dtype: float64

In [48]:
# 대응표본 t-검정을 실행합니다.(양측검정)
pg.ttest(x = df1['before'], y = df1['after'], paired = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.53666,29,two-sided,0.000006,"[0.63, 1.38]",0.102172,3529.131,0.084186


In [49]:
# 대응표본 t-검정을 실행합니다.(단측검정)
pg.ttest(x = df1['before'], y = df1['after'], paired = True, alternative = 'greater')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.53666,29,greater,0.000003,"[0.7, inf]",0.102172,7058.261,0.136056


### 분산분석: FuelType

In [50]:
# FuelType 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'FuelType', method = 'shapiro')

,W,pval,normal
FuelType,,,
Diesel,0.955971,4.250351e-04,False
Petrol,0.978165,5.503549e-12,False
CNG,0.952408,4.957061e-01,True


In [51]:
# (정규성 가정 만족) FuelType 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'FuelType', method = 'levene')

,W,pval,equal_var
levene,6.497893,0.001557,False


In [52]:
# 등분산 가정된 분산분석을 실행합니다.
pg.anova(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,ddof2,F,p-unc,np2
0,FuelType,2,1265,3.982748,0.018869,0.006257


In [53]:
# 이분산 가정된 분산분석을 실행합니다.
pg.welch_anova(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,ddof2,F,p-unc,np2
0,FuelType,2,38.941548,2.611953,0.086207,0.006257


In [54]:
# (정규성 가정 불만족) 크루스칼-왈리스 순위합 검정을 실행합니다.
pg.kruskal(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,H,p-unc
Kruskal,FuelType,2,10.47472,0.005314


### 사후분석(다중비교)

In [55]:
# Bonferroni 방법으로 사후분석을 실행합니다.
sp.posthoc_ttest(a = df, val_col = 'Price', group_col = 'FuelType', 
                 p_adjust = 'bonferroni')

,Diesel,Petrol,CNG
Diesel,1.000000,0.016843,1.0
Petrol,0.016843,1.000000,1.0
CNG,1.000000,1.000000,1.0


In [56]:
# Tukey 방법으로 사후분석을 실행합니다.
sp.posthoc_tukey(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.000000,0.015915,0.900000
Petrol,0.015915,1.000000,0.759308
CNG,0.900000,0.759308,1.000000


In [57]:
# Scheffe 방법으로 사후분석을 실행합니다.
sp.posthoc_scheffe(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.000000,0.022116,0.933882
Petrol,0.022116,1.000000,0.797953
CNG,0.933882,0.797953,1.000000


In [58]:
# Tamhane 방법으로 사후분석을 실행합니다.
sp.posthoc_tamhane(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.000000,0.075664,0.987404
Petrol,0.075664,1.000000,0.931849
CNG,0.987404,0.931849,1.000000


In [59]:
# Nemenyi 방법으로 사후분석을 실행합니다.
sp.posthoc_nemenyi(a = df, val_col = 'Price', group_col = 'FuelType')

,CNG,Diesel,Petrol
CNG,1.000000,0.954287,0.675617
Diesel,0.954287,1.000000,0.007117
Petrol,0.675617,0.007117,1.000000


### 교차분석

In [60]:
# 두 범주형 변수의 빈도수를 출력합니다.
pd.crosstab(index = df['MetColor'], columns = df['Automatic'])

Automatic,0,1
MetColor,,
0,402,26
1,796,44


In [61]:
# 두 범주형 변수의 상대도수를 출력합니다.
pd.crosstab(index = df['MetColor'], columns = df['Automatic'], 
            normalize = 'index', margins = True)

Automatic,0,1
MetColor,,
0,0.939252,0.060748
1,0.947619,0.052381
All,0.944795,0.055205


In [62]:
# 두 범주형 변수로 교차분석(카이제곱 검정)을 실행합니다.
test = pg.chi2_independence(data = df, x = 'MetColor', y = 'Automatic', 
                            correction = True)

In [63]:
# 교차분석 결과를 확인합니다.
test

(Automatic          0         1
 MetColor                      
 0          404.37224  23.62776
 1          793.62776  46.37224,
 Automatic      0     1
 MetColor              
 0          402.5  25.5
 1          795.5  44.5,
                  test    lambda      chi2  dof      pval    cramer     power
 0             pearson  1.000000  0.237030  1.0  0.626360  0.013672  0.077567
 1        cressie-read  0.666667  0.236104  1.0  0.627034  0.013646  0.077458
 2      log-likelihood  0.000000  0.234308  1.0  0.628348  0.013594  0.077246
 3       freeman-tukey -0.500000  0.233008  1.0  0.629302  0.013556  0.077093
 4  mod-log-likelihood -1.000000  0.231749  1.0  0.630230  0.013519  0.076945
 5              neyman -2.000000  0.229348  1.0  0.632008  0.013449  0.076662)

In [64]:
# 피셔의 정확검정을 실행하면 오즈비와 유의확률을 반환합니다.
# test의 두 번째 원소인 관측값 행렬을 지정합니다.
stats.fisher_exact(table = test[1])

SignificanceResult(statistic=0.8899622641509434, pvalue=0.6947795226271706)

### 모비율 검정

In [65]:
# 단일표본 모비율 검정을 실행합니다.
proportions_ztest(count = 30, nobs = 100, value = 0.2)

(2.182178902359923, 0.029096331741252257)

In [66]:
# 두 표본의 모비율 차이 검정을 실행합니다.
proportions_ztest(count = [100, 150], nobs = [300, 500], value = 0.0)

(0.9847319278346616, 0.3247557654026184)

### 변수 제거 및 외부 파일로 저장

In [67]:
# 가설검정 결과를 반영하여 CC와 Automatic을 삭제합니다.
df = df.drop(columns = ['CC', 'Automatic'])

In [68]:
# FuelType이 'CNG'인 건수가 매우 적고 Petrol, Diesel과의 
# 목표변수 평균에서 유의한 차이가 없으므로 삭제합니다.
df = df[df['FuelType'].ne('CNG')]

In [69]:
# df의 행이름을 초기화합니다.
df = df.reset_index(drop = True)

In [70]:
# df를 z 파일로 저장합니다.
joblib.dump(value = df, filename = 'Used_Cars_Price_Prep.z')

['Used_Cars_Price_Prep.z']

## End of Document